In [6]:
import pandas as pd
import numpy as np

df_2020 = pd.read_csv("data/mhcld_puf_2020.csv")
df_2019 = pd.read_csv("data/mhcld_puf_2019.csv")
df_2018 = pd.read_csv("data/mhcld_puf_2018.csv")
df_2017 = pd.read_csv("data/mhcld_puf_2017.csv")
df_2016 = pd.read_csv("data/mhcld_puf_2016.csv")
df_2015 = pd.read_csv("data/mhcld_puf_2015.csv")
df_2014 = pd.read_csv("data/mhcld_puf_2014.csv")
df_2013 = pd.read_csv("data/mhcld_puf_2013.csv")

pop_df = pd.read_csv("data/nst-est2020.csv")

In [7]:
df_2020 = df_2020.groupby(['STATEFIP', 'YEAR'], as_index=False).sum()
df_2019 = df_2019.groupby(['STATEFIP', 'YEAR'], as_index=False).sum()
df_2018 = df_2018.groupby(['STATEFIP', 'YEAR'], as_index=False).sum()
df_2017 = df_2017.groupby(['STATEFIP', 'YEAR'], as_index=False).sum()
df_2016 = df_2016.groupby(['STATEFIP', 'YEAR'], as_index=False).sum()
df_2015 = df_2015.groupby(['STATEFIP', 'YEAR'], as_index=False).sum()
df_2014 = df_2014.groupby(['STATEFIP', 'YEAR'], as_index=False).sum()
df_2013 = df_2013.groupby(['STATEFIP', 'YEAR'], as_index=False).sum()


merged_df = pd.concat([df_2020, df_2019, df_2018, df_2017, df_2016, df_2015, df_2014, df_2013], ignore_index=True)

In [8]:
merged_df

,STATEFIP,YEAR,AGE,EDUC,ETHNIC,RACE,GENDER,SPHSERVICE,CMPSERVICE,OPISERVICE,...,DEPRESSFLG,ODDFLG,PDDFLG,PERSONFLG,SCHIZOFLG,ALCSUBFLG,OTHERDISFLG,DIVISION,REGION,CASEID
0,1,2020,654103,298445,383348,419066,149315,194320,97959,193889,...,31107,3874,1077,2135,17851,15,10296,585162,292581,1970050155806628
1,2,2020,61875,28653,35131,38568,16112,20674,10651,20674,...,2612,250,98,97,648,1038,763,93033,41348,208808461568552
2,4,2020,2493416,-3456863,-408054,-139765,585530,768452,384551,763587,...,71674,2861,15001,3427,27495,64077,54429,3074672,1537336,7763662112306521
3,5,2020,514749,-697968,176075,126255,117235,154512,78548,153985,...,19271,1582,142,390,8543,4177,5106,542864,232656,1566591578134424
4,6,2020,4219076,-6142185,830729,1902616,1005539,1354571,716073,1343822,...,186694,18407,5878,8138,120339,65534,87450,6142185,2729860,13786414714013096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,51,2013,761786,190583,402254,477076,164390,214993,110775,219398,...,36816,6686,0,13028,22617,0,14239,548495,329097,2208814505816820
371,53,2013,1010328,-1156459,500837,653597,215940,292971,150183,286047,...,49811,2253,850,1925,19180,8046,2885,1331793,591908,2979569873429586
372,55,2013,806559,-880308,371175,451111,146818,191185,104142,183617,...,27939,1106,768,3605,16436,2468,9805,293436,195624,1969491659013450
373,56,2013,107715,43385,52645,69104,25740,32871,16892,33164,...,5554,603,243,1265,870,2,7968,132656,66328,333887492931729


In [9]:
df = merged_df[['STATEFIP', 'YEAR', 'ANXIETYFLG', 'TRAUSTREFLG', 'ADHDFLG', 'BIPOLARFLG', 'DEPRESSFLG',
        'SPHSERVICE', 'CMPSERVICE', 'OPISERVICE', 'RTCSERVICE']]

df['TOT_SERVICE'] = df[['SPHSERVICE', 'CMPSERVICE', 'OPISERVICE', 'RTCSERVICE']].sum(axis=1)

df

C:\Users\farajin1\AppData\Local\Temp/ipykernel_15480/3082231611.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TOT_SERVICE'] = df[['SPHSERVICE', 'CMPSERVICE', 'OPISERVICE', 'RTCSERVICE']].sum(axis=1)


,STATEFIP,YEAR,ANXIETYFLG,TRAUSTREFLG,ADHDFLG,BIPOLARFLG,DEPRESSFLG,SPHSERVICE,CMPSERVICE,OPISERVICE,RTCSERVICE,TOT_SERVICE
0,1,2020,16620,13115,15797,12055,31107,194320,97959,193889,195003,681171
1,2,2020,1522,3324,512,526,2612,20674,10651,20674,20198,72197
2,4,2020,75605,69167,32715,30140,71674,768452,384551,763587,768664,2685254
3,5,2020,9586,12098,2795,6851,19271,154512,78548,153985,153974,541019
4,6,2020,106258,111713,43205,54443,186694,1354571,716073,1343822,1356266,4770732
...,...,...,...,...,...,...,...,...,...,...,...,...
370,51,2013,21422,19027,18967,29164,36816,214993,110775,219398,219398,764564
371,53,2013,18855,31489,7322,21972,49811,292971,150183,286047,295848,1025049
372,55,2013,10992,14090,5093,18133,27939,191185,104142,183617,193001,671945
373,56,2013,3068,3975,1533,2368,5554,32871,16892,33164,33164,116091


In [10]:
state_code_to_name = {
    1: 'AL', 2: 'AK', 4: 'AZ', 5: 'AR', 6: 'CA', 8: 'CO', 9: 'CT', 10: 'DE', 11: 'DC',
    12: 'FL', 13: 'GA', 15: 'HI', 16: 'ID', 17: 'IL', 18: 'IN', 19: 'IA', 20: 'KS',
    21: 'KY', 22: 'LA', 25: 'MA', 26: 'MI', 27: 'MN', 28: 'MS',
    29: 'MO', 30: 'MT', 31: 'NE', 32: 'NV', 34: 'NJ', 35: 'NM', 36: 'NY',
    37: 'NC', 38: 'ND', 39: 'OH', 40: 'OK', 41: 'OR', 42: 'PA', 44: 'RI', 45: 'SC',
    46: 'SD', 47: 'TN', 48: 'TX', 49: 'UT', 50: 'VT', 51: 'VA', 53: 'WA',
    55: 'WI', 56: 'WY', 72: 'PR', 99: 'Other'
}

# Map the state codes to state names
df['STATE'] = df['STATEFIP'].map(state_code_to_name)
df

C:\Users\farajin1\AppData\Local\Temp/ipykernel_15480/1363682350.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['STATE'] = df['STATEFIP'].map(state_code_to_name)


,STATEFIP,YEAR,ANXIETYFLG,TRAUSTREFLG,ADHDFLG,BIPOLARFLG,DEPRESSFLG,SPHSERVICE,CMPSERVICE,OPISERVICE,RTCSERVICE,TOT_SERVICE,STATE
0,1,2020,16620,13115,15797,12055,31107,194320,97959,193889,195003,681171,AL
1,2,2020,1522,3324,512,526,2612,20674,10651,20674,20198,72197,AK
2,4,2020,75605,69167,32715,30140,71674,768452,384551,763587,768664,2685254,AZ
3,5,2020,9586,12098,2795,6851,19271,154512,78548,153985,153974,541019,AR
4,6,2020,106258,111713,43205,54443,186694,1354571,716073,1343822,1356266,4770732,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,51,2013,21422,19027,18967,29164,36816,214993,110775,219398,219398,764564,VA
371,53,2013,18855,31489,7322,21972,49811,292971,150183,286047,295848,1025049,WA
372,55,2013,10992,14090,5093,18133,27939,191185,104142,183617,193001,671945,WI
373,56,2013,3068,3975,1533,2368,5554,32871,16892,33164,33164,116091,WY


In [11]:
# Reshape the DataFrame using pd.melt()
df_long = pd.melt(pop_df, id_vars=['STATE', 'NAME'], var_name='YEAR', value_name='POPESTIMATE')

# Clean up the 'Year' column by extracting the year number from the column names
df_long['YEAR'] = df_long['YEAR'].str.extract('(\d{4})')
df_long['YEAR'] = df_long['YEAR'].astype(int)

df_long

,STATE,NAME,YEAR,POPESTIMATE
0,1,Alabama,2013,4831586
1,2,Alaska,2013,737626
2,4,Arizona,2013,6634690
3,5,Arkansas,2013,2960459
4,6,California,2013,38253768
...,...,...,...,...
463,53,Washington,2020,7693612
464,54,West Virginia,2020,1784787
465,55,Wisconsin,2020,5832655
466,56,Wyoming,2020,582328


In [18]:
data = pd.merge(df, df_long, left_on=['STATEFIP', 'YEAR'], right_on=['STATE', 'YEAR'], how='inner')
data = data.drop('STATE_y', axis=1)
data['UTIL_RATE'] = data['TOT_SERVICE']/data['POPESTIMATE']
data

,STATEFIP,YEAR,ANXIETYFLG,TRAUSTREFLG,ADHDFLG,BIPOLARFLG,DEPRESSFLG,SPHSERVICE,CMPSERVICE,OPISERVICE,RTCSERVICE,TOT_SERVICE,STATE_x,NAME,POPESTIMATE,UTIL_RATE
0,1,2020,16620,13115,15797,12055,31107,194320,97959,193889,195003,681171,AL,Alabama,4921532,0.138406
1,2,2020,1522,3324,512,526,2612,20674,10651,20674,20198,72197,AK,Alaska,731158,0.098743
2,4,2020,75605,69167,32715,30140,71674,768452,384551,763587,768664,2685254,AZ,Arizona,7421401,0.361826
3,5,2020,9586,12098,2795,6851,19271,154512,78548,153985,153974,541019,AR,Arkansas,3030522,0.178523
4,6,2020,106258,111713,43205,54443,186694,1354571,716073,1343822,1356266,4770732,CA,California,39368078,0.121183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,50,2013,3379,7550,2194,2008,5422,48870,24492,48870,46788,169020,VT,Vermont,626603,0.269740
363,51,2013,21422,19027,18967,29164,36816,214993,110775,219398,219398,764564,VA,Virginia,8255861,0.092609
364,53,2013,18855,31489,7322,21972,49811,292971,150183,286047,295848,1025049,WA,Washington,6966252,0.147145
365,55,2013,10992,14090,5093,18133,27939,191185,104142,183617,193001,671945,WI,Wisconsin,5738012,0.117104


In [25]:
trend_df = pd.read_csv("data/pytrends_data.csv")

# Split 'key' column into 'state' and 'year' columns
trend_df[['country_state', 'year']] = trend_df['key'].str.split('_', expand=True)
trend_df['state'] = trend_df['country_state'].str[-2:]  # Extract the state abbreviation from the country_state part
trend_df['year'] = trend_df['year'].astype(int)  # Convert year to integer
trend_df = trend_df.drop(['key', 'country_state', 'Unnamed: 0'], axis=1) #remove uneeded columns

trend_df

,max_ADHD,max_PTSD,max_anxiety,max_bipolar,max_depression,max_mental hospital,max_psychiatrists near me,max_psychologist near me,max_therapist near me,mean_ADHD,...,min_PTSD,min_anxiety,min_bipolar,min_depression,min_mental hospital,min_psychiatrists near me,min_psychologist near me,min_therapist near me,year,state
0,29.0,19.0,57.0,26.0,74.0,0.0,0.0,0.0,0.0,20.583333,...,10.0,39.0,14.0,45.0,0.0,0.0,0.0,0.0,2013,AK
1,34.0,22.0,62.0,27.0,78.0,64.0,0.0,0.0,0.0,21.250000,...,14.0,41.0,15.0,46.0,0.0,0.0,0.0,0.0,2014,AK
2,27.0,26.0,68.0,24.0,92.0,0.0,0.0,99.0,0.0,20.000000,...,13.0,44.0,9.0,44.0,0.0,0.0,0.0,0.0,2015,AK
3,28.0,22.0,76.0,30.0,89.0,68.0,0.0,0.0,0.0,22.500000,...,11.0,53.0,15.0,55.0,0.0,0.0,0.0,0.0,2016,AK
4,28.0,21.0,90.0,23.0,90.0,0.0,0.0,0.0,0.0,21.333333,...,12.0,63.0,11.0,57.0,0.0,0.0,0.0,0.0,2017,AK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,29.0,18.0,79.0,21.0,82.0,0.0,0.0,60.0,81.0,22.166667,...,10.0,58.0,15.0,47.0,0.0,0.0,0.0,0.0,2020,WY
608,42.0,20.0,79.0,32.0,65.0,0.0,0.0,0.0,79.0,34.083333,...,0.0,65.0,13.0,40.0,0.0,0.0,0.0,0.0,2021,WY
609,54.0,18.0,82.0,26.0,70.0,77.0,0.0,0.0,74.0,42.750000,...,10.0,60.0,15.0,43.0,0.0,0.0,0.0,0.0,2022,WY
610,65.0,18.0,82.0,23.0,73.0,79.0,0.0,0.0,77.0,49.583333,...,6.0,65.0,17.0,39.0,0.0,0.0,0.0,0.0,2023,WY


In [28]:
#merge health data & trend data on year & state
data_1 = pd.merge(data, trend_df, left_on=['STATE_x', 'YEAR'], right_on=['state', 'year'], how='inner')
data_1 = data_1.drop('year', axis=1) #remove uneeded columns
data_1

,STATEFIP,YEAR,ANXIETYFLG,TRAUSTREFLG,ADHDFLG,BIPOLARFLG,DEPRESSFLG,SPHSERVICE,CMPSERVICE,OPISERVICE,...,min_PTSD,min_anxiety,min_bipolar,min_depression,min_mental hospital,min_psychiatrists near me,min_psychologist near me,min_therapist near me,year,state
0,1,2020,16620,13115,15797,12055,31107,194320,97959,193889,...,12.0,72.0,18.0,58.0,29.0,0.0,0.0,25.0,2020,AL
1,2,2020,1522,3324,512,526,2612,20674,10651,20674,...,8.0,67.0,15.0,63.0,0.0,0.0,0.0,0.0,2020,AK
2,4,2020,75605,69167,32715,30140,71674,768452,384551,763587,...,13.0,74.0,18.0,58.0,29.0,0.0,11.0,30.0,2020,AZ
3,5,2020,9586,12098,2795,6851,19271,154512,78548,153985,...,15.0,75.0,18.0,56.0,28.0,0.0,0.0,29.0,2020,AR
4,6,2020,106258,111713,43205,54443,186694,1354571,716073,1343822,...,13.0,84.0,16.0,63.0,37.0,0.0,6.0,29.0,2020,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,50,2013,3379,7550,2194,2008,5422,48870,24492,48870,...,9.0,37.0,13.0,41.0,0.0,0.0,0.0,0.0,2013,VT
346,51,2013,21422,19027,18967,29164,36816,214993,110775,219398,...,9.0,48.0,20.0,46.0,25.0,0.0,0.0,0.0,2013,VA
347,53,2013,18855,31489,7322,21972,49811,292971,150183,286047,...,11.0,51.0,19.0,49.0,36.0,0.0,0.0,0.0,2013,WA
348,55,2013,10992,14090,5093,18133,27939,191185,104142,183617,...,7.0,46.0,18.0,48.0,29.0,0.0,0.0,0.0,2013,WI


In [31]:
data_1.to_csv('data/aggregated_data.csv', index=False)